# Set up your environment

- The following example will work in your R Studio session with the packages indicated.  

- In case you want to follow along in Jupyter:

    Provided you have Jupyter installed (if not, you can download Anaconda), please follow the instructions at- https://irkernel.github.io/installation/  

    Then on the conda prompt run:
    
    <code>jupyter-notebook</code>

In [89]:
# Loading required libraries

# Below I may explicitly call functions to indicate where they're from where there may arise ambiguity

success <- suppressWarnings(sapply(c('pacman'), require, character.only = TRUE))
if (length(names(success)[!success]) == 0) cat('pacman available') else install.packages(names(success)[!success])

pacman::p_load(foreach, doParallel, future, rsample, dplyr)
pacman::p_load_gh(c("DavisVaughan/furrr"))

pacman available

# What does high performance computing involve?

Particularly for the R ecosystem, you can find a list of packages and resources here- https://cran.r-project.org/web/views/HighPerformanceComputing.html

# Illustration of doParallel and foreach

Our first example will use these two packages. On Windows we stick to <code>makeCluster</code> instead of <code>makeForkCluster</code> from the <code>parallel</code> package. If you're not on windows <code>mclapply</code> also uses forking to allow lapply functionality parallelly.

Alternatives to <code>doParallel</code> are <code>doMPI</code>, <code>doSNOW</code>, etc.

In [125]:
# Register a parallel backend

cl <- makeCluster(parallel:::detectCores() - 1) # one fewer than what's available for safety
registerDoParallel(cl)

### Numerical Riemann integration example from *Advanced R*

https://en.wikipedia.org/wiki/Newton%E2%80%93Cotes_formulas

In [91]:
newton_cotes <- function(coef, open = FALSE) {
  n <- length(coef) + open

  function(f, a, b) {
    pos <- function(i) a + i * (b - a) / n
    points <- pos(seq.int(0, length(coef) - 1))

    (b - a) / sum(coef) * sum(f(points) * coef)
  }
}


monte_carlo <- function(f, a, b, n) {
    x <- runif(n, a, b)
    y <- (f(x)) / (1 / (b - a))
    integral <- sum(y)/n
    
    integral
}    

    
composite <- function(f, a, b, n, rule) {
  if (deparse(substitute(rule)) != 'monte_carlo') {
      points <- seq(a, b, length = n + 1)

      area <- 0
      for (i in seq_len(n)) {
        area <- area + rule(f, points[i], points[i + 1])
      }

      area
  } else {
      monte_carlo(f, a, b, n)
  }
}
    



boole <- newton_cotes(c(7, 32, 12, 32, 7))
composite(sin, 0, pi, n = 1000, rule = boole)

composite(sin, 0, pi, n = 1000, rule = monte_carlo)


[1] 2

[1] 1.957866

### Lessons

- A functional idiom helps us avoid passing around data to workers.

In [100]:
paramlist <- list(trapezoid = c(1, 1), simpson = c(1, 4, 1), simpson2 = c(1, 3, 3, 1), boole = c(7, 32, 12, 32, 7),
                  midpoint = c(c(1), open = TRUE), milne = c(c(2, -1, 2), open = TRUE),
                  willm = c(c(11, 1, 1, 11), open = TRUE))

integrals <- foreach (i = seq_along(paramlist), .combine = c) %do% {
    newton_cotes_rule <- newton_cotes(paramlist[[i]])
    value <- setNames(composite(sin, 0, pi, n = 100000, rule = newton_cotes_rule), names(paramlist)[i])
}

integrals
mean(integrals)

trapezoid   simpson  simpson2     boole  midpoint     milne     willm 
        2         2         2         2         2         2         2

[1] 2

### Exercise: Bagging

In [102]:
data('attrition')
contrasts(attrition$Attrition)

p <- 3/4
indices <- sample(nrow(attrition), size = floor((nrow(attrition) * p)))  
trainset <- attrition[indices,]
testset <- attrition[-indices,]

lm_fit <- glm(Attrition ~ JobSatisfaction + Gender + MonthlyIncome + YearsSinceLastPromotion,
              data = trainset, family = binomial)
summary(lm_fit)

probabilities <- predict(lm_fit, testset, type = 'response')
predictions <- rep('No', dim(testset)[1])
predictions[probabilities > 1/2] <- 'Yes'
table(predictions, testset[['Attrition']])

,Yes
No,0
Yes,1



Call:
glm(formula = Attrition ~ JobSatisfaction + Gender + MonthlyIncome + 
    YearsSinceLastPromotion, family = binomial, data = trainset)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-0.9795  -0.6528  -0.5432  -0.3361   2.8184  

Coefficients:
                          Estimate Std. Error z value Pr(>|z|)    
(Intercept)             -9.814e-01  1.874e-01  -5.237 1.63e-07 ***
JobSatisfaction.L       -6.356e-01  1.661e-01  -3.826  0.00013 ***
JobSatisfaction.Q        1.002e-01  1.699e-01   0.590  0.55545    
JobSatisfaction.C       -2.066e-01  1.733e-01  -1.192  0.23312    
GenderMale               1.151e-01  1.727e-01   0.666  0.50519    
MonthlyIncome           -1.367e-04  2.682e-05  -5.098 3.43e-07 ***
YearsSinceLastPromotion  2.736e-02  2.949e-02   0.928  0.35353    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 977.87  on 1101  degrees of freedom
Residual dev

           
predictions  No Yes
         No 310  58

In [126]:
# helper function for mode
modefun <- function(x) {
  ux <- unique(x)
  ux[which.max(tabulate(match(x, ux)))]
}


bagged_lm_fit <- function(dataset, trainset, testset, quotient = 4, iterations = 10000) {
    
    predictions <- foreach(m=1:iterations, .combine=cbind) %dopar% {
        train_indices <- sample(nrow(trainset), size = floor((nrow(trainset) / quotient)))
        
        lm_fit <- glm(Attrition ~ JobSatisfaction + Gender + MonthlyIncome + YearsSinceLastPromotion,
                     data = dataset[(1:nrow(trainset) %in% train_indices),], family = binomial)
        
        probabilities <- predict(lm_fit, newdata = testset, type = 'response')
        predictions <- rep('No', dim(testset)[1])
        predictions[probabilities > 1/2] <- 'Yes'
        predictions
        }
    
    apply(predictions, 1, modefun)

}

# Let's compare dopar vs do:
starttime <- proc.time()
# You can also use the microbenchmark package or tictoc

results <- bagged_lm_fit(attrition, trainset, testset)

duration <- (endtime <- proc.time())[3] - starttime[3]
duration

elapsed 
  23.82

### Redo with furrr

In [127]:
# To get you started:

plan(multiprocess)

quotient <- 4
iterations <- 1:1000

indices <- purrr::map(iterations, ~sample(nrow(trainset), size = floor((nrow(trainset) / quotient))))

In [128]:
# Remember to stop your cluster once done.

stopCluster(cl)